In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma

#Chroma Faiss는 성능차이로 내가 골라 쓰면됨
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA


llm = ChatOpenAI()

cache_dir = LocalFileStore("../.cache/")


splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("../data/hong.pdf")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

#vectorstore = FAISS.from_documents(docs, cached_embeddings)
vectorstore = Chroma.from_documents(docs, cached_embeddings)


cahin = RetrievalQA.from_chain_type(
    llm = llm,
    #chain_type = "refine",
    #chain_type = "retrieval",
    chain_type = "map_reduce",
    retriever = vectorstore.as_retriever(),

    )

cahin.run("홍길동을 묘사해줘!")


# embedder = OpenAIEmbeddings()

# #vector = embedder.embed_query("Hi! yoon. How are you?")
# vector = embedder.embed_documents([
#     "hi",
#     "how",
#     "are",
#     "you longer sentenses because"
# ])

# print(len(vector),len(vector[0]))

Created a chunk of size 935, which is longer than the specified 600


'홍길동은 한국의 전래동화나 소설에 나오는 전설적인 인물로, 대표적인 영웅으로 여겨지며 의리, 정의, 용기 등의 가치를 상징하는 인물입니다. 총명하면서도 재주 많고 용감한 모습으로 묘사되며, 어떤 상황에서도 사람들을 돕고 불의에 맞서는 용기를 보여주는 인물로 유명합니다. 주변에 있는 사람들을 도와주는 성품을 지니고 있으며, 유머 감각이 풍부하고 똑똑한 면도 갖추고 있습니다. 홍길동은 중국 소설과 한국 전래동화에서 등장하는 인물로, 정의로운 성품과 용감한 행동으로 유명한 인물입니다.'

In [9]:
result = vectorstore.similarity_search("홍길동은 어머니는 어때?")
print(result)

[Document(page_content='이루지 못했다. 하루는 어머니 침소에 가 길동이 울면서 말했다.\n홍길동전\uf07c허균\n“소자, 어머님과 더불어 전생의 연분이 귀중하여 금세(今世)에 모자지간(母子之間)이\n되었으니 은혜가 망극하옵니다. 그러나 소자의 팔자가 기박(奇薄)하여 천한 몸이 되오니\n품은 한이 깊사옵니다. 장부가 세상에 살면서 남의 천대를 받는 것이 당치 않은 일이라,\n소자가 자연히 기운을 억제하지 못하여 어머님 슬하를 떠나려 하옵니다. 엎드려\n바라오니, 어머님은 소자를 염려하지 마시고 귀한 몸을 보중하옵소서.”', metadata={'source': '../data/hong.pdf'}), Document(page_content='이루지 못했다. 하루는 어머니 침소에 가 길동이 울면서 말했다.\n홍길동전\uf07c허균\n“소자, 어머님과 더불어 전생의 연분이 귀중하여 금세(今世)에 모자지간(母子之間)이\n되었으니 은혜가 망극하옵니다. 그러나 소자의 팔자가 기박(奇薄)하여 천한 몸이 되오니\n품은 한이 깊사옵니다. 장부가 세상에 살면서 남의 천대를 받는 것이 당치 않은 일이라,\n소자가 자연히 기운을 억제하지 못하여 어머님 슬하를 떠나려 하옵니다. 엎드려\n바라오니, 어머님은 소자를 염려하지 마시고 귀한 몸을 보중하옵소서.”', metadata={'source': '../data/hong.pdf'}), Document(page_content='길동이 두 번 절하고 하직하니, 공이 붙들지 못하고 다만 무사하기만을 당부했다.\n길동은 또 어머니 침소에 가 이별을 고했다.\n“소자가 지금 슬하를 떠나고자 하옵니다. 다시 모실 날이 있을 것이오니 어머님은\n그사이 귀중한 몸을 보중하소서.”\n춘섬이 이 말을 듣고 무슨 변괴가 있었음을 짐작하여, 하직하는 아들을 보고 손을\n잡고 통곡하며 말했다.\n“네 어디로 향하고자 하느냐? 한집에 있어도 처소가 너무 멀어 매양 그리워하더니,\n이제 너를 정처 없이 보내고 어찌 살겠느냐? 너는 쉬이 돌아와 